In [1]:
from langchain.output_parsers.pydantic import PydanticOutputParser
from pydantic import BaseModel, Field

In [2]:
class Response(BaseModel):
    answer: str = Field(description="answer to the user's question")
    source: str = Field(description="source used to answer the user's question, should be a website.")

In [3]:
from langchain.prompts import PromptTemplate

In [4]:
output_parser = PydanticOutputParser(response_schema=Response)

In [5]:
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=["question"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [6]:
from langchain.llms import OpenAI

In [7]:
model = OpenAI(temperature=0)

In [10]:
foo = model(prompt.format(question="what's the capital of paris"))

In [12]:
output_parser.parse(foo)

Response(answer='Paris is the capital of France.', source='https://www.worldatlas.com/articles/what-is-the-capital-of-france.html')